In [8]:
pip install arviz

In [9]:
import pystan
import numpy as np
import matplotlib.pyplot as plt
import arviz
import json
import pandas as pd

In [10]:
with open('/content/drive/MyDrive/TFM-Crowd learning/multinomial.json') as json_file:
    data = json.load(json_file)

In [11]:
data.keys()

dict_keys(['w', 't', 'a', 'k', 't_A', 'w_A', 'ann'])

In [12]:
print('Un total de '+str(data['w'])+ ' workers')
print('Un total de '+str(data['t'])+ ' tasques')
print('Un total de '+str(data['a'])+' anotacions')
print('Ja que k = '+str(data['k'])+', vol dir que els workers han de triar entre 3 possibles anotacions. Per aquest motiu, la key "ann" només conté 0, 1 o 2.')

Un total de 20 workers
Un total de 1000 tasques
Un total de 10000 anotacions
Ja que k = 3, vol dir que els workers han de triar entre 3 possibles anotacions. Per aquest motiu, la key "ann" només conté 0, 1 o 2.


In [13]:
dades={'Tasca': data['t_A'], 'Worker': data['w_A'], 'Anotació': data['ann'] }

In [14]:
df = pd.DataFrame(data=dades)
df

,Tasca,Worker,Anotació
0,0,14,2
1,0,13,2
2,0,4,2
3,0,2,2
4,0,7,1
...,...,...,...
9995,999,17,2
9996,999,2,2
9997,999,13,2
9998,999,14,2


In [15]:
sm2 = pystan.StanModel(file='/content/drive/MyDrive/TFM-Crowd learning/Fitxers STAN/Multinomial.fit_and_consensus.stan')


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_eb22c1a71e18909250dd5ae3d1ee0693 NOW.


In [16]:
w = len(np.unique(df['Worker']))
t = len(np.unique(df['Tasca']))
a = len(df['Anotació'])
k = len(np.unique(df['Anotació']))
t_A = np.array(df['Tasca'])+1
w_A = np.array(df['Worker'])+1
ann = np.array(df['Anotació'])+1
tau_prior = np.array([1,1,1])
pi_prior = np.ones((k,k))

dades_sm2 = {'w':w,'t':t,'a':a,'k':k,'t_A':t_A,'w_A':w_A,'ann':ann,'tau_prior':tau_prior,'pi_prior':pi_prior} 

In [17]:
pi_ini = np.eye(3)+0.05
pi_ini /= np.sum(pi_ini,axis=1)
print(pi_ini)
dades_inicials={'pi':pi_ini}

[[0.91304348 0.04347826 0.04347826]
 [0.04347826 0.91304348 0.04347826]
 [0.04347826 0.04347826 0.91304348]]


In [18]:
sm2.optimizing(data=dades_sm2,init=dades_inicials)

OrderedDict([('tau', array([0.11447569, 0.29653163, 0.58899268])),
             ('pi', array([[0.68294039, 0.10402746, 0.21303215],
                     [0.21831892, 0.59436455, 0.18731653],
                     [0.09335233, 0.10484681, 0.80180086]])),
             ('log_p_t_C', array([[-18.34730299, -16.81046269,  -4.77265154],
                     [-10.19255143, -15.73836315, -19.82600751],
                     [-17.89912662, -15.50261261,  -8.95749087],
                     ...,
                     [-22.1998555 ,  -8.57445457, -21.1636503 ],
                     [-16.46555033, -17.81199862,  -4.88877106],
                     [-17.63051484, -17.9651557 ,  -2.73829163]])),
             ('t_C', array([[1.27233262e-06, 5.91618737e-06, 9.99992811e-01],
                     [9.96046419e-01, 3.88833929e-03, 6.52413237e-05],
                     [1.30622057e-04, 1.43485935e-03, 9.98434519e-01],
                     ...,
                     [1.20937659e-06, 9.99995382e-01, 3.40862917e-06]